In this code the goal is to build a pipeline to create preprocess the mri images

In [2]:
import SimpleITK as sitk
import numpy as np
import os

In [3]:
DATA_PARENT_DIR = '../Task1/brain/'
dataset_dir = [os.path.join(DATA_PARENT_DIR, mri_path, 'mr.nii.gz') for mri_path in os.listdir(DATA_PARENT_DIR)]
mask_dir = [os.path.join(DATA_PARENT_DIR, mri_path, 'mask.nii.gz') for mri_path in os.listdir(DATA_PARENT_DIR)]
category = [dataset_dir[i].split('/')[-2][2] for i in range(len(dataset_dir))]

## BBox

In [5]:
def get_bbox(image):
    bbox=[]

    labelimfilter=sitk.LabelShapeStatisticsImageFilter()
    labelimfilter.Execute(image)
    for i in range(1,labelimfilter.GetNumberOfLabels()+1):
        box=labelimfilter.GetBoundingBox(i)
        bbox.append(box)

    if len(bbox) == 0:
        return None
    else:
        return bbox[0]


In [21]:
image_sitk = sitk.ReadImage(dataset_dir[0])
image = sitk.GetArrayFromImage(image_sitk)
# mask = sitk.GetArrayFromImage(sitk.ReadImage(mask_dir[0]))
mask = sitk.ReadImage(mask_dir[0])
# get_bbox(mask)
labelimfilter=sitk.LabelShapeStatisticsImageFilter()
labelimfilter.Execute(mask)
labelimfilter.GetNumberOfLabels()
labelimfilter.GetBoundingBox(1), image_sitk.GetSize()


((10, 10, 0, 193, 227, 181), (212, 246, 181))

In [11]:
mask

NameError: name 'mask' is not defined